source

kagle  (https://www.kaggle.com/datasets/blastchar/telco-customer-churn/download?datasetVersionNumber=1)

github (https://github.com/Briankimany/data-science/blob/main/week%201/churn.csv)

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix



/home/brian/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-10-01 15:47:34.230258: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 15:47:39.253982: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 15:47:39.266995: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 15:47:49.756171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from keras.layers import Dropout
from keras import regularizers
from keras.callbacks import ModelCheckpoint

###### RELEVANT COLUMNS
> CreditScore—can have an effect on customer churn, since a customer with a higher credit score is less likely to leave the bank.

> Geography—a customer’s location can affect their decision to leave the bank.

> Gender—it’s interesting to explore whether gender plays a role in a customer leaving the bank.

> Age—this is certainly relevant, since older customers are less likely to leave their bank than younger ones.

> Tenure—refers to the number of years that the customer has been a client of the bank. Normally, older clients are more loyal and less likely to leave a bank.

> Balance—also a very good indicator of customer churn, as people with a higher balance in their accounts are less likely to leave the bank compared to those with lower balances.

> HasCrCard—denotes whether or not a customer has a credit card. This column is also relevant, since people with a credit card are less likely to leave the bank.

> IsActiveMember—active customers are less likely to leave the bank.

> EstimatedSalary—as with balance, people with lower salaries are more likely to leave the bank compared to those with higher salaries.

> Exited (1(y), o(n))

In [3]:
data = pd.read_csv("/home/brian/PycharmProjects/allison learing/churn.csv")
data_1 = pd.read_csv("/home/brian/PycharmProjects/allison learing/churn.csv")
unused_list = ['RowNumber',	'CustomerId',	'Surname',	'NumOfProducts']
new_data = data_1
#exited = data.pop("Exited")
for i in unused_list:
    new_data.pop(i)


new_data = new_data.sample(frac= 1 , random_state=2)
new_data = pd.get_dummies(new_data)


def Scaler(new_data , ratio = 9000):
    train_data = new_data[:ratio]
    test_data = new_data[ratio:]
    train_data_val = train_data.pop('Exited')
    test_data_val = test_data.pop("Exited")

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train_data.to_numpy())
    Y_train = train_data_val.to_numpy()
    X_test_scaled = scaler.fit_transform(test_data.to_numpy())
    Y_test = test_data_val.to_numpy()
    return X_train_scaled , Y_train , X_test_scaled , Y_test


In [71]:
def r_g(X_train_scaled , y_train , X_test_scaled , y_test , PLOT = False):
    forest_base = RandomForestRegressor(n_estimators=50, max_depth=5).fit(X_train_scaled, y_train)

    #print (forest_base_test_predictions)
    return forest_base

def LR(X_train_scaled , y_train , X_test_scaled , y_test , PLOT = False , history = ''):
    model = LogisticRegression()

    model.fit(X_train_scaled, y_train)
    
    return model

def compute_metrics(y_pred , PLOT ,y_test = None ):
    accuracy = accuracy_score(y_test , (y_pred >0.5 ).astype(int) )
    precision = precision_score(y_test, (y_pred >0.5 ).astype(int))
    recall = recall_score(y_test ,(y_pred >0.5 ).astype(int))
    f1 = f1_score(y_test,(y_pred >0.5 ).astype(int))
    roc_auc = roc_auc_score(y_test, (y_pred >0.5 ).astype(int))
    
    mae = mean_absolute_error(y_test, y_pred)


    print (("Here are the results:\nAccuracy {}\nPrecission {}\nRecall {}\nF1 score {}\nroc_acc {}\nThe MAe{}\nconf_matrix {}\n{}").format(accuracy , precision,recall , f1 ,roc_auc ,mae, conf_matrix , sum(conf_matrix[0])))


    return mae

def Conf_matrix(y_pred):
    y_pred_binary = (y_pred > 0.5).astype(int)
    cm = confusion_matrix(y_test, y_pred_binary)  # y_test should be your true labels
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", annot_kws={"size": 16})
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
 



def neural_net(X_train_scaled , y_train , PLOT = False , num_epochs = 15 , input_shape =12 ):
    # Build the neural network
    
   

    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(input_shape,), kernel_regularizer=regularizers.l2(0.01)),
        keras.layers.Dropout(0.5),  # Dropout layer with a 50% dropout rate
        keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        keras.layers.Dropout(0.5),  # Dropout layer with a 50% dropout rate
        keras.layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

   

    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
    #history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[checkpoint])


    # Training
    history = model.fit(X_train_scaled, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2 ,  callbacks=[checkpoint])


    if PLOT:
        training_loss = history.history['loss']
        training_accuracy = history.history['accuracy']
        validation_loss = history.history['val_loss']
        validation_accuracy = history.history['val_accuracy']

        # Plot the training and validation loss
        plt.plot(training_loss, label='Training Loss')
        plt.plot(validation_loss, label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

        # Plot the training and validation accuracy
        plt.plot(training_accuracy, label='Training Accuracy')
        plt.plot(validation_accuracy, label='Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

    # Evaluation
    return model
    #loss, accuracy = model.evaluate(X_test_scaled, y_test)


def pca_trans(X_train , X_test , n_comp = 10):
    from sklearn.decomposition import PCA
    shape_1 = X_train.shape
    pca = PCA(n_components=n_comp)
    pca = pca.fit(X_train)

    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    shape_2 = X_train_pca.shape
    print (f"INPUT DATA SHAPE WAS {shape_1}  final shape = {shape_2}")
    return X_train_pca , X_test_pca
